In [ ]:
from keras import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

train_csv = pd.read_csv("Data/train.csv")
train_csv_2 = train_csv.drop(columns=['target'])
print(train_csv_2.shape)
train_csv_2 = train_csv_2.append(pd.read_csv("Data/test_independent.csv"))
train_csv_one_hot = pd.get_dummies(train_csv_2)

train_csv_one_hot

scaled_data = MinMaxScaler().fit_transform(train_csv_one_hot)
print(scaled_data.shape)

model = Sequential()

enc = Sequential()
enc.add(Dense(300, activation='relu'))
enc.add(Dense(150, activation='relu'))
enc.add(Dense(10, name='embeddings', activation='relu'))

dec = Sequential()
dec.add(Dense(100, activation='relu'))
dec.add(Dense(150, activation='relu'))
dec.add(Dense(102, name='output'))


model.add(enc)
model.add(dec)

model.compile(optimizer='adam', loss='mse')
model.fit(scaled_data, scaled_data, epochs=5)

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

encodings = enc.predict(scaled_data)

pca = PCA()
principalComponents = pca.fit_transform(encodings)
print(pca.explained_variance_ratio_)
x_val = []
y_val= []
for i in range(principalComponents.shape[0]):
    x_val.append(principalComponents[i][0])
    y_val.append(principalComponents[i][1])
x_val = np.array(x_val)
y_val = np.array(y_val)

plt.scatter(x=x_val, y=y_val, alpha=0.5)

plt.legend(labels=np.arange(0,9))
plt.show()
from sklearn.cluster import AgglomerativeClustering

n_clusters = 2
cluster = AgglomerativeClustering(n_clusters=n_clusters, 
                                  affinity='euclidean',
                                  linkage='ward')
cl_nom = cluster.fit_predict(principalComponents)
targets = pd.read_csv("Data/train.csv")['target']
print(np.sum(abs(cl_nom[:13741] - targets )))
pred = pd.read_csv("Data/Pred.csv")['target']

print(np.sum(abs(cl_nom[13741:] - pred)))